# PPO（近端策略算法）
[Proximal Policy Optimization Algorithms](https://arxiv.org/abs/1707.06347)



## 基础概念
![强化学习概念](img/rl.png)

- action space: 可选择的动作
  - {$\uparrow, \rightarrow, \downarrow, \leftarrow$} 记为action  {$a_1, a_2, a_3, a_4$}

- Policy: 策略函数，输入state 输出 action 的概率分布
  - $\pi(a_1| s_t ) = 0.1$
  - $\pi(a_2| s_t ) = 0.5$

- Trajectory：轨迹一般用 $\tau$ 表示表示一连串的状态-动作序列。
  - $ \tau \thicksim {s_1, a_2, s_2, a_3, s_5, ...}$
  - 状态转移：
    - 确定： $s_{t + 1} = f(s_t, a_t)$
    - 不确定：$s_{t +1 } = f(\cdot|s_t,a_t)$

- Return：回报，从当前时间点到游戏结束的reward的累计和。
 - 最大最后收益retuern，而非当前的reward

## 采样

$$\mathrm{E}(x)_{x \thicksim p(x)} = \sum _{x} x p(x) \approx \frac{1}{N} x{ _{x_i \thicksim p(x_i)}}$$

目标，训练一个Policy神经网路$\pi$，在所有状态state下，给出相应的action和得到的return
$$
E(R(\tau))_{\tau \thicksim P_{\theta}(\tau)} = \sum_{\tau}R(\tau)P_{\theta}(\tau)\\
$$
$\theta$ 是Policy网络的参数， $\tau$ trajectory也是由神经网络决定的。

我们使用梯度上升的方法最大化return
$$
\begin{aligned}
\nabla E(R(\tau))_{\tau \thicksim P_{\theta}(\tau)} &= \nabla \sum_{\tau}R(\tau)P_{\theta}(\tau)\\
&= \sum_{\tau}R(\tau) \nabla P_{\theta}(\tau)\\
&= \sum_{\tau}R(\tau) \nabla P_{\theta}(\tau) \frac{P_{\theta}(\tau)}{P_{\theta}(\tau)}\\
&= \sum_{\tau}R(\tau) P_{\theta}(\tau) \frac{\nabla P_{\theta}(\tau)}{P_{\theta}(\tau)}\\
&\approx \frac{1}{N} \sum_{n}^{N} R(\tau^n) \frac{\nabla P_{\theta}(\tau^n)}{P_{\theta}(\tau^n)} \quad \text{n是假标号 且} \nabla log f(x) = \frac{\nabla f(x)}{f(x)} \\
&= \frac{1}{N} \sum_{n}^{N} R(\tau^n)\nabla log P_{\theta}(\tau^n) \quad \text{其中} \tau \thicksim P_{\theta}(\tau)\\
&=\frac{1}{N} \sum_{n}^{N} R(\tau^n)\nabla \prod_{t = 1}^{T_n} log P_{\theta}(a_n^t|s_n^t)\quad \text{假设下一状态是由当前状态和动作决定}\\
&= \frac{1}{N} \sum_{n}^{N} R(\tau^n) \sum_{t = 1}^{T_n} \nabla log P_{\theta}(a_n^t|s_n^t) \\
&= \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) \\
\end{aligned}
$$
从最后的表达式来看是：  
采样得到的轨迹 $\tau^{(n)}$ 就能得到性能目标函数 
$$J(\theta) = E_{\tau \thicksim P}[R(\tau)]$$
对策略参数 $\theta$ 的梯度 $\nabla_{\theta} J(\theta)$  
用这条梯度做策略改进： $$\theta \leftarrow \theta + \nabla_{\theta} J(\theta)$$

## 训练神经网络
$$ 
loss = - \frac{1}{N}\sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) 
$$ 
负号是因为我们要最小化loss

$$ 
$$

![训练](img/on%20policy.png)
经过神经网络训练后softmax得到 $P(a_n^t | s_n^t)$ 的概率，  
接着我们让神经网络玩n场游戏，得到n个trajectory，和对应的reward也即 $R(\tau^n)$  
这样就可以进行batch的训练更新神经网络，这样往复循环。  
但是这延伸出一个问题，我们大部分时间都花在采样上了，训练非常慢。

**首先考虑的是reward的改进**

原式是跟据一整条trajectory的reward来更新概率的，但是在做出一个action后按照经验是影响后续的reward，并且影响更大的是靠近action之后的reward。为此引入一个折扣因子（discount factor, $\gamma$）：
$$R(\tau^n) \rightarrow \sum_{t' = t} ^{T_n} \gamma^{t' - t}r_{t'}^{n} = R^n_t$$
表示尽可能用当前的动作对trajectory的retuen影响。
$$
\frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) = \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R^n_t  \nabla log P_{\theta}(a_n^t|s_n^t) 
$$


并且我们希望在相对好的局势下增加概率，我们引入actor-critis的baesline：
$$
\begin{aligned}
&\frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R(\tau^n)  \nabla log P_{\theta}(a_n^t|s_n^t) \\
&= \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} R^n_t  \nabla log P_{\theta}(a_n^t|s_n^t) \\
&= \frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} (R^n_t - B(s_t^n))  \nabla log P_{\theta}(a_n^t|s_n^t) 
\end{aligned}
$$

![](img/a2c.png)

## More definition

- Action-Value Function  
$R^n_t$每次都是一次随机采样，方差很大训练不稳定
$Q_{\theta}(s, a)$ 在state 下，做出action-a，期望的回报，称为动作价值函数。  

- State-Value FUnction   
$V_{\theta}(s)$ 在state-s下，期望的回报，称为状态价值函数

- Advantage Function  
$A_{\theta} = Q_{\theta}(s, a) -V_{\theta}(s)$ 在state -s下做出action -a，比其他动作能带来多少的优势。


有了优势函数的概念我们将公式优化为：  
$$
\frac{1}{N} \sum_{n}^{N} \sum_{t = 1}^{T_n} A_{\theta}(s_t^n, a_t^n) \nabla log P_{\theta}(a_n^t|s_n^t) 
$$
由 $Q_{\theta}(s, a) = r_t + \gamma V_{\theta}(s_{t+1})$ 带入：
$$
A_{\theta}(s_t^n, a_t^n) = r_t + \gamma V_{\theta}(s_{t+1}) - V_{\theta}(s_t)
$$
这样神经网络简化为只需要拟合状态价值函数（State-Value FUnction）


对状态价值函数也进行采样：
$$V_{\theta}(s_{t+1}) \approx r_{r+1} + \gamma V_{\theta}(s_{t+2})$$
* 贝尔曼方程得来

这样我们可以对(s,a)进行：  
一步采样 $A_{\theta}^{1}(s_t^n, a_t^n) = r_t + \gamma V_{\theta}(s_{t+1}) - V_{\theta}(s_t)$\
两步采样 $A_{\theta}^{2}(s_t^n, a_t^n) = r_t + \gamma r_{t+1}V_{\theta}(s_{t+2}) - V_{\theta}(s_t)$\
...\
全采样 $A_{\theta}^{T}(s_t^n, a_t^n) = r_t + \gamma^2 r_{t+3} + ... +\gamma^T r_T - V_{\theta}(s_t)$\
采样越多方差越多偏差越小，为了简介表示定义：
$$
\delta_t^V =  r_t + \gamma V_{\theta}(s_{t+1}) - V_{\theta}(s_t)
$$
转化为：  
$A_{\theta}^{1}(s_t^n, a_t^n) = \delta^V_t$\
$A_{\theta}^{2}(s_t^n, a_t^n) = \delta^V_t + \gamma\delta^V_{t+1}$\
$A_{\theta}^{2}(s_t^n, a_t^n) = \delta^V_t + \gamma\delta^V_{t+1} + \gamma^2\delta^V_{t+2}$
...


## Generalized Advantage Estimation (GAE)
$$
\begin{align*}
A_{\theta}^{GAE}(s_t, a) &= (1 - \lambda)(A_{\theta}^1 + \lambda \cdot A_{\theta}^2 + \lambda^2 A_{\theta}^3 + \cdots) \\
&= (1 - \lambda)(\delta_t^V + \lambda \cdot (\delta_t^V + \gamma \delta_{t+1}^V) + \lambda^2 (\delta_t^V + \gamma \delta_{t+1}^V + \gamma^2 \delta_{t+2}^V) + \cdots) \\
&= (1 - \lambda)(\delta_t^V (1 + \lambda + \lambda^2 + \cdots) + \gamma \delta_{t+1}^V \cdot (\lambda + \lambda^2 + \cdots) + \cdots) \\
&= (1 - \lambda)(\delta_t^V \frac{1}{1 - \lambda} + \gamma \delta_{t+1}^V \frac{\lambda}{1 - \lambda} + \cdots) \\
&= \sum_{b=0}^{\infty} (\gamma \lambda)^b \delta_{t+b}^V
\end{align*}
$$
例子：
$$
\begin{align*}
\lambda = 0.9: \quad A_{\theta}^{GAE} = 0.1 A_{\theta}^1 + 0.09 A_{\theta}^2 + 0.081 A_{\theta}^3 + \cdots
\end{align*}
$$

最后我们的到三个最终式子：
![values function](img/vlues%20function.png)

通过这个神经网络来拟合价值函数--当前步到结束的衰减加和作为label。

## on-policy and off-policy
- on-policy：学习所用的数据必须由当前正在优化的策略产生。
  - 边采样、边更新；每次更新后旧数据作废	
  - REINFORCE、A2C/A3C
- off-policy：学习所用的数据可以来自任何策略（通常是旧策略或行为策略）。
  - 可把旧经验反复回放（replay buffer）
  - 需要 importance sampling（重要性采样） or repaly buffer（经验放回池）

重要性采样：
$$
\begin{align*}
\mathbb{E}(f(x))_{x \sim p(x)} &= \sum_x f(x) \cdot p(x) \\
&= \sum_x f(x) \cdot p(x) \frac{q(x)}{q(x)} \\
&= \sum_x f(x) \frac{p(x)}{q(x)} \cdot q(x) \\
&= \mathbb{E}\left(f(x) \frac{p(x)}{q(x)}\right)_{x \sim q(x)}\\
& \approx \frac{1}{N} \sum{N}{n} \frac{p(x_n)}{q(x_n)}f(_n) 
\end{align*}
$$
把难以计算/采样效率低的p分布转换为q分布

使用参考策略 $\theta'$ 更新训练策略 $\theta$

$$
\begin{aligned}
&\frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_n} A_{\theta}^{GAE}(s_n^t, a_n^t) \nabla \log P_{\theta}(a_n^t | s_n^t)\quad \nabla \log f(x) = \frac{\nabla f(x)}{f(x)}\\\\

&= \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_n} A_{\theta'}^{GAE}(s_n^t, a_n^t) \frac{P_{\theta}(a_n^t | s_n^t)}{P_{\theta'}(a_n^t | s_n^t)} \nabla \log P_{\theta}(a_n^t | s_n^t)\\

&= \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_n} A_{\theta'}^{GAE}(s_n^t, a_n^t) \frac{P_{\theta}(a_n^t | s_n^t)}{P_{\theta'}(a_n^t | s_n^t)} \frac{\nabla P_{\theta}(a_n^t | s_n^t)}{P_{\theta}(a_n^t | s_n^t)}\\

&= \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_n} A_{\theta'}^{GAE}(s_n^t, a_n^t) \frac{\nabla P_{\theta}(a_n^t | s_n^t)}{P_{\theta}(a_n^t | s_n^t)}\\

\text{Loss} &= - \frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_n} A_{\theta'}^{GAE}(s_n^t, a_n^t) \frac{P_{\theta}(a_n^t | s_n^t)}{P_{\theta'}(a_n^t | s_n^t)}\\

\end{aligned}
$$

- 优化这个损失函数意味着我们希望增加采取那些具有高优势的动作的概率，同时减少采取那些具有低优势的动作的概率。
- $\frac{P_{\theta}(a_n^t | s_n^t)}{P_{\theta'}(a_n^t | s_n^t)}$ 称为重要性采样比率(Importance Sampling Ratio)它用于校正使用旧策略 $\theta'$ 生成的数据来更新新策略 $\theta$ 时的偏差。这是 off-policy 学习的一个关键特性。



$$
\text{Loss}_{\text{PPO}} = -\frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_n} A_{\theta'}^{GAE}(s_n^t, a_n^t) \frac{P_{\theta}(a_n^t | s_n^t)}{P_{\theta'}(a_n^t | s_n^t)} + \beta \text{KL}(P_{\theta}, P_{\theta'})
$$
$$
\begin{aligned}
\text{Loss}_{\text{PPO2}}&\\&= -\frac{1}{N} \sum_{n=1}^{N} \sum_{t=1}^{T_n} \min \left( A_{\theta'}^{GAE}(s_n^t, a_n^t) \frac{P_{\theta}(a_n^t | s_n^t)}{P_{\theta'}(a_n^t | s_n^t)}, 
\text{clip} \left( \frac{P_{\theta}(a_n^t | s_n^t)}{P_{\theta'}(a_n^t | s_n^t)}, 1 - \epsilon, 1 + \epsilon \right) A_{\theta'}^{GAE}(s_n^t, a_n^t) \right)
\end{aligned}
$$

# 补充内容
## KL 散度项的作用（Loss<sub>PPO</sub> 中的 KL 惩罚）


### 1. 目的  
在 **PPO-Penalty**（又称 PPO-KL）里，**KL(π_θ || π_θ_old)** 充当**“信任域”**的角色，限制新策略 π_θ 与旧策略 π_θ_old 之间的距离，防止在一次更新中步子迈得太大，导致策略崩溃。  
这种做法继承自 TRPO（Trust Region Policy Optimization），但 TRPO 用硬约束（二次近似 + 共轭梯度），而 PPO 用**软约束**——把 KL 直接写进目标函数，变成带系数的惩罚项。

### 2. 数学形式  

$$\mathcal{L}^{\text{KL}}(\theta)=
\mathbb{E}_{(s,a)\sim\pi_{\theta_{\text{old}}}}
\Bigl[
\frac{\pi_\theta(a|s)}{\pi_{\theta_{\text{old}}}(a|s)}
A^{\text{GAE}}_{\theta_{\text{old}}}(s,a)
\Bigr]
- \beta\,
\mathbb{E}_{s\sim\pi_{\theta_{\text{old}}}}\Bigl[
D_{\text{KL}}\bigl(\pi_\theta(\cdot|s)\big\|\pi_{\theta_{\text{old}}}(\cdot|s)\bigr)
\Bigr]$$


- 第一项是**策略梯度信号**，希望提高高优势动作的概率；  
- 第二项是**KL 惩罚**，系数 β 按如下规则自适应调整：  
  - 若 KL < δ/1.5 → β ← β/2 (步子可再大点)  
  - 若 KL > δ·1.5 → β ← β·2 (步子需再小点)  
  - 否则 β 不变  

### 3. 参考 
- John Schulman et al., “**High-Dimensional Continuous Control Using Generalized Advantage Estimation**”, 2016 [arXiv:1506.02438](https://arxiv.org/abs/1506.02438) —— 提出 GAE 与早期 KL 惩罚思想。  
- John Schulman et al., “**Proximal Policy Optimization Algorithms**”, 2017 ([arXiv:1707.06347](https://arxiv.org/abs/1707.06347)) —— PPO 原文第 3.1 节。  
- 中文教材：博育 AI 强化学习课程 [第 12 章 PPO 算法](https://hrl.boyuai.com/chapter/2/ppo%E7%AE%97%E6%B3%95/)。


## Clip 截断是什么（Loss<sub>PPO2</sub> 中的 clip）


### 1. 目的  
在 **PPO-Clip**（学术界/工业界更常用的版本）里，**不再显式计算 KL**，而是直接在目标函数里对**概率比**  
$$
r_t(\theta)=\frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}
$$
做硬截断，从而**隐式地**把策略更新约束在一个“可信区域”内。

### 2. 数学形式  
$$
\mathcal{L}^{\text{CLIP}}(\theta)=
\mathbb{E}_{(s,a)\sim\pi_{\theta_{\text{old}}}}
\Bigl[
\min\!\bigl(
r_t(\theta)\,A_t,\;
\text{clip}(r_t(\theta),\,1-\varepsilon,\,1+\varepsilon)\,A_t
\bigr)
\Bigr]
$$

- 当 $(r_t)$ 落在 $([1-\varepsilon, \, 1+\varepsilon])$ 之外时，**第二项立即变成常数**，对该样本的梯度为 0，从而**阻止进一步拉大/缩小概率**。  
- 经验上 $(\varepsilon=0.1\sim0.2)$ 即可有效抑制策略震荡。

### 3. 实现细节  
- 一个 batch 的经验可以重复利用 `ppo_epochs` 次，每次都用上述 **clip 目标**更新，而不必重新采样。  
- 当 \(r_t\) 被 clip 时，停止该样本对 Actor 的梯度贡献（即“**梯度切断**”），避免过度优化。

### 4. 参考
- Schulman et al., 2017 PPO 原文第 3.2 节 “Clipped Surrogate Objective” [arXiv:1707.06347](https://arxiv.org/abs/1707.06347)。  
- 中文博客：知乎「人人都能看懂的 PPO 原理与源码解读」；  
- GitHub 开源实现：llm_note 中 RLHF-PPO 章节。


## 总结

- **KL 惩罚** = 显式地把“新旧策略距离”写进目标函数，用拉格朗日乘子动态调整；  
- **Clip 截断** = 隐式地把距离限制在区间 $([1-\varepsilon,\,1+\varepsilon])$ 内，实现更简单、效果更好，已成为 PPO 的默认实现。

两者都服务于同一目标：**在提升策略性能的同时，避免一次更新步子过大导致训练崩塌**。

1. TRPO 

TRPO 把策略更新抽象成一个 **带 KL 约束的期望回报最大化**：
$$\begin{aligned}
&\max_\theta \; \mathbb{E}_{s\sim\rho_{\theta_{\text{old}}},\,a\sim\pi_{\theta_{\text{old}}}} 
\!\!\Bigl[ \frac{\pi_\theta(a|s)}{\pi_{\theta_{\text{old}}}(a|s)} A_{\theta_{\text{old}}}(s,a) \Bigr] \\[4pt]
&\text{s.t.} \quad \mathbb{E}_{s\sim\rho_{\theta_{\text{old}}}}\!\Bigl[ D_{\text{KL}}\!\bigl(\pi_\theta(\cdot|s)\big\|\pi_{\theta_{\text{old}}}(\cdot|s)\bigr)\Bigr] \le \delta.
\end{aligned}
$$
- 目标：最大化**改进下界**（surrogate objective）。  
- 约束：新策略与旧策略的平均 KL 不超过阈值 $(\delta)$。
TRPO 用共轭梯度 + 线性搜索来硬满足这条约束，实现较复杂。

2. PPO-Penalty：把约束改成罚函数

为了简化实现，PPO-Penalty 把 KL 约束**拉格朗日化**，变成**可微的罚函数**：
$$
\mathcal{L}^{\text{KL}}(\theta)=
\underbrace{
\mathbb{E}\!\left[
\frac{\pi_\theta}{\pi_{\theta_{\text{old}}}} A_{\theta_{\text{old}}}
\right]}_{\text{要最大化的项}}
- \beta\,
\underbrace{
\mathbb{E}\!\left[
D_{\text{KL}}(\pi_\theta\|\pi_{\theta_{\text{old}}})
\right]}_{\text{KL 罚函数}}
$$

- 如果我们要 **最大化** $(\mathcal{L}^{\text{KL}})$，自然把罚函数写成 **减号**（因为 KL ≥ 0）。  
- 在大多数深度学习框架里，优化器默认做 **最小化**，于是代码里再把整个式子乘以 **−1**：
$$
\text{Loss}^{\text{KL}}(\theta) = -\mathcal{L}^{\text{KL}}(\theta)
= -\mathbb{E}\!\left[\frac{\pi_\theta}{\pi_{\theta_{\text{old}}}} A_{\theta_{\text{old}}}\right]
+ \beta\,
\mathbb{E}\!\left[D_{\text{KL}}(\pi_\theta\|\pi_{\theta_{\text{old}}})\right]
$$


## 其他参考

[ppo in hugging face](https://huggingface.co/blog/deep-rl-ppo)

[ppo in pytorch](https://docs.pytorch.org/tutorials/intermediate/reinforcement_ppo.html)

[PPO Explained: The RL Algorithm That Took the World by Storm](https://medium.com/@vivek_tiwari_vt/ppo-explained-the-rl-algorithm-that-took-the-world-by-storm-8a245910b8ef)

[Understanding the Mathematics of PPO in Reinforcement Learning](https://towardsdatascience.com/understanding-the-mathematics-of-ppo-in-reinforcement-learning-467618b2f8d4/)